### 0 - Configuración

In [ ]:
!pip install sagemaker python-dotenv --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.7/844.7 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.5 MB/s eta 0:00:00


In [ ]:
import sagemaker
import boto3

import os
from dotenv import load_dotenv

### 1 - Sesión AWS

In [ ]:
# variables de entorno
# Opción 1
# os.environ["aws_access_key_id"]='aws_access_key_id'
# os.environ["aws_secret_access_key"]='aws_secret_access_key'

# Opción 2
load_dotenv()

True

  [Secret keys](https://docs.aws.amazon.com/powershell/latest/userguide/pstools-appendix-sign-up.html)

In [ ]:
REGION_NAME = "us-east-1"
os.environ["AWS_DEFAULT_REGION"] = REGION_NAME
ROLE_NAME =  'Sagemaker-ExecutionRole'

auth_arguments = {
    'aws_access_key_id':os.environ["aws_access_key_id"],
    'aws_secret_access_key':os.environ["aws_secret_access_key"],
    'region_name':REGION_NAME
}

[IAM rol](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)

In [ ]:
iam = boto3.client('iam', **auth_arguments)
role = iam.get_role(RoleName=ROLE_NAME)['Role']['Arn']

session = sagemaker.Session(boto3.Session(**auth_arguments))

### 2 - Despliegue

In [ ]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# image uri
llm_image = get_huggingface_llm_image_uri("huggingface")

print(f"image uri: {llm_image}")

image uri: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.0-tgi0.8.2-gpu-py39-cu118-ubuntu20.04


In [ ]:
from sagemaker.huggingface import HuggingFaceModel

# Falcon 7b
hub = {'HF_MODEL_ID':'tiiuae/falcon-7b'}

# Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,
   role=role,  # iam role from AWS
   image_uri=llm_image,
   sagemaker_session=session
)

In [ ]:
# deploy model to SageMaker
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.g5.2xlarge', #'ml.g5.4xlarge'
 	container_startup_health_check_timeout=300
)

--------------!

### 3 - Inferencia

In [ ]:
# define prompt
prompt = """You are the most advanced AI assistant on the planet, called Falcon.

User: What is the capital of Spain? Think step by step.
Falcon:"""

# hyperparameters for llm
request = {
  "inputs": prompt,
  "parameters": {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.7,
    "max_new_tokens": 512,
    "stop": ["\nUser:","<|endoftext|>","</s>"]
  }
}

# request to endpoint
response = predictor.predict(request)

# model response
assistant = response[0]["generated_text"][len(prompt):]

In [ ]:
print(assistant)

 The capital of Spain is Madrid.
User:


In [ ]:
# BORRAR ENDPOINT para evitar gastos innecesarios
predictor.delete_model()
predictor.delete_endpoint()